In [ ]:
import hashlib

# 建立雜湊值
def _md5(string):
    '''
    說明：
    建立雜湊機制

    用法：
    m = hashlib.md5()              # 建立 MD5 物件
    str = 'yourValue'              # 建立想要雜湊的值
    m.update(str.encode("utf-8"))  # 更新 MD5 雜湊值
    strHash = m.hexdigest()        # 取得 MD5 雜湊值
    '''
    m = hashlib.md5()
    m.update(string.encode("utf-8"))
    strKey = m.hexdigest()
    return strKey 

# 原始資料
strOrigin = 'Hello World!'

# 輸出原始 strOrigin
print("修改前資料: " + strOrigin)

# 用 md5 雜湊 strOrigin
print("修改前資料的雜湊結果: " +  _md5(strOrigin) )

# 印出分隔線
print("=" * 55)

# 突然有人把 strOrigin 改了
strOrigin = 'Show me the money!'

# 輸出修改後的 strOrigin
print("修改後資料: " + strOrigin)

# 再次用 md5 雜湊改掉後的 strOrigin
print("修改後資料的雜湊結果: " +  _md5(strOrigin) )

In [ ]:
import requests, re, pprint, os

# 自訂標頭
my_headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'
}

# 將自訂標頭加入 GET 請求中
response = requests.get('https://store.line.me/stickershop/product/17555/zh-Hant', headers = my_headers)

# 建立儲存圖片的資料夾，不存在就新增
folderPath = 'line_stickers'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)
    
#放貼圖資訊用
listLineStickers = [] 

# 正規表達式
pattern = r'https?:\/\/stickershop\.line-scdn\.net\/stickershop\/v1\/sticker\/([0-9]+)\/android\/sticker\.png'


'''
以下為一般做法
'''

# 取得 match 結果
matchIter = re.finditer(pattern, response.text)

# 若有比對到資料 (不為 None)，則執行主要程式
if matchIter != None:
    # 建立 set，以協助去除重複
    s = set()

    # 迭代取得 full match 與各別的 group
    for match in matchIter:
        # 將每一個 match 到的 full match 與 group 資訊，建立成 dict
        myDict = {
            "link": match.group(),
            "id": match.group(1)
        }

        '''
        一、dict.items()
        說明:
            items() 方法把字典中每一對 key 和 value 組成一個 tuple
        例如:
            dict_items([
                ('link', 'https://stickershop.line-scdn.net/stickershop/v1/sticker/318800558/android/sticker.png'), 
                ('id', '318800558')
            ])

        二、tuple(dict.items())
        說明: 
            1. 將 dict_items 格式轉成 tuple，目前是為了讓 set 可以使用 .add() 方法，來去除重複。
            2. 之所以要將轉換格式，是因為 tuple 可以被新增到 set 當中，dict 和 dict_items 不行. 
            3. tuple 是可以雜湊的(hashable)，可雜湊代表「雜湊值不可變動」，不可變動才能拿來判斷是否相同或比較（equal or compare）。
            4. 可變動的資料型態，例如 list 可以 append()、remove()，或是像 dict 等透過指定 key 來新增修改、刪除資料的格式。
        例如:
            (
                ('link','https://stickershop.line-scdn.net/stickershop/v1/sticker/318800558/android/sticker.png'),
                ('id', '318800558')
            )
        '''
        # 將 dict 轉成 dict_items 格式，再轉換成 tuple 格式
        myTuple = tuple(myDict.items()) 

        # 將 tuple 新增到 set 當中
        s.add(myTuple)

    # 新增空的 list，方便之後用 for 迴圈各別下載
    listLineStickers = []
    
    # 將 set 當中的 tuple，逐個轉成原先的 dict，即可達到去除重複的效果
    for t in s:
        '''
        三、dict(t)
        說明:
            原先的 tuple(dict.items()) 的結果，透過 dict() 轉型，變成原先 dict 的 key-value 格式
        例如:
            {
                'id': '318800558',
                'link': 'https://stickershop.line-scdn.net/stickershop/v1/sticker/318800558/android/sticker.png'
            }
        '''
        # 將 tuple 轉成 dict
        myDict = dict(t)
        
        # 將 set 去除重複後的所有 dict，各別新增在 list 當中
        listLineStickers.append(myDict)

    # 將 list 中的 dict 依 id 進行排序
    listLineStickers = sorted(listLineStickers, key=lambda myDict: myDict['id'], reverse=False)
    
    # 輸出已經去除重覆、重新排序的 list 內容
    pprint.pprint(listLineStickers)

    # 下載圖片
    for obj in listLineStickers: 
        os.system(f"curl {obj['link']} -o {folderPath}/{obj['id']}.png")
        
'''
最快做法
matchIter = re.finditer(pattern, response.text)
if matchIter != None:
    for match in matchIter:
        listLineStickers.append({
            "link": match.group(),
            "id": match.group(1)
        })
    listLineStickers = [dict(t) for t in set(tuple(myDict.items()) for myDict in listLineStickers)]
    listLineStickers = sorted(listLineStickers, key=lambda myDict: myDict['id'], reverse=False)
    for obj in listLineStickers: 
        os.system(f"curl {obj['link']} -o {folderPath}/{obj['id']}.png")
'''